In [1]:
!python -V

Python 3.11.5


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('/Users/aasth/Desktop/Data '
 'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1700705746970, experiment_id='1', last_update_time=1700705746970, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = pd.read_parquet('/Users/aasth/Desktop/Data analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/data/yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('/Users/aasth/Desktop/Data analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/data/yellow_tripdata_2022-02.parquet')

In [9]:
len(df_train), len(df_val)

(2463931, 2979431)

In [10]:
df_train.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [11]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds() / 60)

df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_train[categorical] = df_train[categorical].astype(str)

In [12]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_val[categorical] = df_val[categorical].astype(str)

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.530264593247959

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "mlops-zoomcamp/data/yellow_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path", "mlops-zoomcamp/data/yellow_tripdata_2022-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
    mlflow.sklearn.log_model(lr, artifact_path="models_pickle")
    

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [23]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=15,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [25]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [22:48:16] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.24851                          
[1]	validation-rmse:5.37029                          
[2]	validation-rmse:5.13924                          
[3]	validation-rmse:5.06214                          
[4]	validation-rmse:5.03360                          
[5]	validation-rmse:5.01800                          
[6]	validation-rmse:5.01002                          
[7]	validation-rmse:5.00579                          
[8]	validation-rmse:5.00234                          
[9]	validation-rmse:4.99975                          
[10]	validation-rmse:4.99339                         
[11]	validation-rmse:4.99141                         
[12]	validation-rmse:4.98919                         
[13]	validation-rmse:4.98675                         
[14]	validation-rmse:4.98090                         
 20%|██        | 1/5 [01:16<05:05, 76.46s/trial, best loss: 4.980898226623643]

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [22:49:32] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.18051                                                   
[1]	validation-rmse:8.70752                                                   
[2]	validation-rmse:8.28432                                                   
[3]	validation-rmse:7.90698                                                   
[4]	validation-rmse:7.57033                                                   
[5]	validation-rmse:7.27146                                                   
[6]	validation-rmse:7.00626                                                   
[7]	validation-rmse:6.77203                                                   
[8]	validation-rmse:6.56503                                                   
[9]	validation-rmse:6.38297                                                   
[10]	validation-rmse:6.22127                                                  
[11]	validation-rmse:6.07994                                                  
[12]	validation-rmse:5.95604                        

...

[0]	validation-rmse:5.65591                                                    
[1]	validation-rmse:5.18290                                                    
[2]	validation-rmse:5.09593                                                    
[3]	validation-rmse:5.06885                                                    
[4]	validation-rmse:5.06062                                                    
[5]	validation-rmse:5.05584                                                    
[6]	validation-rmse:5.05110                                                    
[7]	validation-rmse:5.04284                                                    
[8]	validation-rmse:5.03918                                                    
[9]	validation-rmse:5.03604                                                    
[10]	validation-rmse:5.03248                                                   
[11]	validation-rmse:5.02825                                                   
[12]	validation-rmse:5.02330            

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [22:52:57] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.53289                                                   
[1]	validation-rmse:7.63813                                                   
[2]	validation-rmse:6.96630                                                   
[3]	validation-rmse:6.46660                                                   
[4]	validation-rmse:6.09990                                                   
[5]	validation-rmse:5.83102                                                   
[6]	validation-rmse:5.63424                                                   
[7]	validation-rmse:5.49022                                                   
[8]	validation-rmse:5.38446                                                   
[9]	validation-rmse:5.30540                                                   
[10]	validation-rmse:5.24587                                                  
[11]	validation-rmse:5.19984                                                  
[12]	validation-rmse:5.16424                        

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [22:54:19] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.88521                                                   
[1]	validation-rmse:6.75768                                                   
[2]	validation-rmse:6.08401                                                   
[3]	validation-rmse:5.68813                                                   
[4]	validation-rmse:5.45540                                                   
[5]	validation-rmse:5.31642                                                   
[6]	validation-rmse:5.22952                                                   
[7]	validation-rmse:5.17586                                                   
[8]	validation-rmse:5.13939                                                   
[9]	validation-rmse:5.11293                                                   
[10]	validation-rmse:5.09534                                                  
[11]	validation-rmse:5.08195                                                  
[12]	validation-rmse:5.07240                        

In [26]:
mlflow.xgboost.autolog(disable=True)

In [27]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=15,
        evals=[(valid, 'validation')],
        early_stopping_rounds=5
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [23:02:41] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.10506
[1]	validation-rmse:8.57550
[2]	validation-rmse:8.11213
[3]	validation-rmse:7.70772
[4]	validation-rmse:7.35607
[5]	validation-rmse:7.05077
[6]	validation-rmse:6.78632
[7]	validation-rmse:6.55873
[8]	validation-rmse:6.36232
[9]	validation-rmse:6.19376
[10]	validation-rmse:6.04900
[11]	validation-rmse:5.92480
[12]	validation-rmse:5.81780
[13]	validation-rmse:5.72615
[14]	validation-rmse:5.64758


/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [23:03:47] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
#run id of our model
logged_model = 'runs:/2dabc4596bf64f43a0590490745b0597/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [23:39:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 2dabc4596bf64f43a0590490745b0597

In [8]:
#Load model as an xgboost model.
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [23:40:06] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [16]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog(disable=True)

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)
        print("Done fitting " + model_class)
        y_pred = mlmodel.predict(X_val)
        print("Done predicting " + model_class)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        print("Done logging " + model_class)
        